In [2]:
from __future__ import print_function
from __future__ import division

import pandas as pd
import numpy as np
import urllib.request
import zipfile
import random
import itertools
import math
import os
import datetime
import sys
import random

In [3]:
# !pip3 install pyshp
# !pip3 install descartes
# !pip3 install xgboost
# !pip3 install bayesian_optimization

import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.model_selection import train_test_split, cross_val_score
import socket
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from bayes_opt import bayesian_optimization

from random import randint
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

import statsmodels.api as sm
import sklearn.model_selection as cv
from scipy import stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import seaborn as sns


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [0]:
'''
# !pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth 
from oauth2client.client import GoogleCredentials

#Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
'''

In [0]:
'''
link = 'https://drive.google.com/open?id=1cQyoWmsTiiWCnlzHeiJ5dtPIRETiHPuo'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('jan-2017-2018-2019-combined.csv')
'''

# Import Data

In [6]:
data = pd.read_csv('dataset/network_traffic_data.csv', delimiter = ',')

In [7]:
for col in data.columns: 
    print(col) 

frame.number
frame.time
wlan.addr
wlan.ta
wlan.ra
wlan.sa
wlan.da
wlan.bssid
wlan.fc.type
wlan.fc.type_subtype
radiotap.channel.freq
radiotap.datarate
radiotap.dbm_antsignal


In [10]:
data.describe(include = 'all')

,frame.number,frame.time,wlan.addr,wlan.ta,wlan.ra,wlan.sa,wlan.da,wlan.bssid,wlan.fc.type,wlan.fc.type_subtype,radiotap.channel.freq,radiotap.datarate,radiotap.dbm_antsignal
count,1.102862e+06,1102861,1102861,886316,1102861,868241,868253,867011,1.102861e+06,1.102861e+06,1102861.0,1.102861e+06,1.102861e+06
unique,NaN,1102861,16938,6532,16938,6620,16636,140,NaN,NaN,NaN,NaN,NaN
top,NaN,"Nov 7, 2019 10:18:45.415474448 +04",ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,NaN,NaN,NaN,NaN,NaN
freq,NaN,1,579223,584094,579223,483942,596066,639848,NaN,NaN,NaN,NaN,NaN
mean,2.524026e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.509643e-01,1.591533e+01,2457.0,1.220855e+00,-4.852092e+01
std,2.852816e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.652723e-01,1.274262e+01,0.0,8.437218e-01,1.217564e+01
min,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,2457.0,1.000000e+00,-9.400000e+01
25%,7.090000e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,5.000000e+00,2457.0,1.000000e+00,-5.100000e+01
50%,1.521500e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,8.000000e+00,2457.0,1.000000e+00,-4.600000e+01
75%,3.157000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,2.900000e+01,2457.0,1.000000e+00,-3.900000e+01


In [9]:
data.head()

,frame.number,frame.time,wlan.addr,wlan.ta,wlan.ra,wlan.sa,wlan.da,wlan.bssid,wlan.fc.type,wlan.fc.type_subtype,radiotap.channel.freq,radiotap.datarate,radiotap.dbm_antsignal
0,1,"Nov 10, 2019 17:44:56.905180894 +04",ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,0.0,8.0,2457.0,1.0,-45.0
1,2,"Nov 10, 2019 17:44:57.090171784 +04",00:1b:b1:ab:7b:af,34:97:f6:ac:9c:10,00:1b:b1:ab:7b:af,34:97:f6:ac:9c:10,00:1b:b1:ab:7b:af,34:97:f6:ac:9c:10,2.0,44.0,2457.0,1.0,-47.0
2,3,"Nov 10, 2019 17:44:57.824486813 +04",ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,0.0,8.0,2457.0,1.0,-46.0
3,4,"Nov 10, 2019 17:44:59.155412559 +04",ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,0.0,8.0,2457.0,1.0,-46.0
4,5,"Nov 10, 2019 17:45:00.179597000 +04",ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,ff:ff:ff:ff:ff:ff,34:97:f6:ac:9c:10,0.0,8.0,2457.0,1.0,-46.0
